In [ ]:
import serial
import time
import pandas as pd
import serial.tools.list_ports
from datetime import datetime

# Check available ports and print them
ports = serial.tools.list_ports.comports()
for port in ports:
    print(port)

# Initialize serial connection (adjust COM port and baud rate as needed)
try:
    ser = serial.Serial('COM4', 115200, timeout=1)  # Replace 'COM3' with your serial port
except serial.SerialException as e:
    print(f"Error: {e}")
    exit(1)

# Initialize a list to store the data
data = []

# Read data from serial
def read_serial_data(duration=60):
    start_time = time.time()
    while time.time() - start_time < duration:
        if ser.in_waiting > 0:
            try:
                line = ser.readline().decode('utf-8', errors='ignore').strip()
                if line:
                    print(line)  # Print the raw line for debugging
                    # Parse the line to extract values
                    if "River Code:" in line and "Post Code:" in line and "Temperature:" in line and "Humidity:" in line and "Distance:" in line and "Rain:" in line:
                        try:
                            parts = line.split(',')
                            post_code = parts[0].split(':')[1].strip()
                            river_code = parts[1].split(':')[1].strip()
                            t = float(parts[2].split(':')[1].strip().replace('C', '').strip())
                            h = float(parts[3].split(':')[1].strip().replace('%', '').strip())
                            distance = float(parts[4].split(':')[1].strip().replace('cm', '').strip())
                            rainValue = float(parts[5].split(':')[1].strip())
                            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                            data.append({'Timestamp': timestamp, 'River Code': river_code, 'Post Code': post_code, 'Temperature': t, 'Humidity': h, 'Distance': distance, 'RainValue': rainValue})
                        except ValueError as ve:
                            print(f"Invalid data received: {ve}")
                    else:
                        print("Incorrect data format received")
            except UnicodeDecodeError as e:
                print(f"Decode error: {e}")
        time.sleep(1)

# Read data for 60 seconds
read_serial_data(10)

print(data)

# Convert the data to a DataFrame
sensor_b = pd.DataFrame(data)

# Display the DataFrame
print(sensor_b.head())


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import array
from numpy import hstack
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import pymysql
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error
from math import sqrt
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
import random, sqlite3

In [ ]:
timestamp = sensor_b['Timestamp'].iloc[0]
post_code = sensor_b['Post Code'].iloc[0]
river_code = sensor_b['River Code'].iloc[0]
temperature = sensor_b['Temperature'].iloc[0]
humidity = sensor_b['Humidity'].iloc[0]
distance = sensor_b['Distance'].iloc[0]
rain_value = sensor_b['RainValue'].iloc[0]

In [ ]:
print(timestamp)
print(post_code)
print(river_code)
print(temperature)
print(humidity)
print(distance)
print(rain_value)

In [ ]:
# Database connection
db_name = "floodeck_alpha"
db_host = "localhost"
db_username = "root"
db_password = ""

try:
    conn = pymysql.connect(host=db_host,
                           port=int(3306),
                           user=db_username,
                           password=db_password,
                           db=db_name)
    print("Connection successful")
except Exception as e:
    print(e)

In [ ]:
sensor_placement_query = "SELECT post_sensor_placement FROM water_post_data WHERE post_code = %s"

In [ ]:
sensor_placement = pd.read_sql_query(sensor_placement_query, conn, params=[post_code])
sensor_placement = sensor_placement.astype(float).iloc[0, 0]
sensor_placement

In [ ]:
river_depth_query = "SELECT river_depth FROM river_data WHERE river_code = %s"

In [ ]:
river_depth = pd.read_sql_query(river_depth_query, conn, params=[river_code])
river_depth = river_depth.astype(float).iloc[0, 0]
river_depth

In [ ]:
Sensor_y_scan_water_height = distance - sensor_placement

sensor_b_scan_water_height = river_depth - Sensor_y_scan_water_height

# sensor_b_scan_water_height = river_safe_height.astype(float)

sensor_b_scan_water_height

In [ ]:
river_safe_height_query = "SELECT river_safe_height FROM river_data WHERE river_code = %s"

In [ ]:
river_safe_height = pd.read_sql_query(river_safe_height_query, conn, params=[river_code])
river_safe_height = river_safe_height.astype(float).iloc[0, 0] * river_depth
river_safe_height

In [ ]:
river_warning_height_query = "SELECT river_warning_height FROM river_data WHERE river_code = %s"

In [ ]:
river_warning_height = pd.read_sql_query(river_warning_height_query, conn, params=[river_code])
river_warning_height = river_warning_height.astype(float).iloc[0, 0] * river_depth
river_warning_height

In [ ]:
river_danger_height_query = "SELECT river_danger_height FROM river_data WHERE river_code = %s"

In [ ]:
river_danger_height = pd.read_sql_query(river_danger_height_query, conn, params=[river_code])
river_danger_height = river_danger_height.astype(float).iloc[0, 0] * river_depth
river_danger_height

In [ ]:
if sensor_b_scan_water_height >= river_danger_height:
    point_b_status = "DANGER"
    print("DANGER")
elif sensor_b_scan_water_height >= river_warning_height:
    point_b_status = "WARNING"
    print("WARNING")
elif sensor_b_scan_water_height >= river_safe_height:
    point_b_status = "SAFE"
    print("SAFE")
else:
    point_b_status = "SAFE"
    print("SAFE")

In [ ]:
# sql_query = "INSERT INTO dhtsensor (sensor_post_code, dht_humidity, dht_temperature, created_at) VALUES (%s, %s, %s, %s)"

# print(sql_query)

# try:
#     with conn.cursor() as cursor:

#         cursor.execute(sql_query, (post_code, humidity, temperature, timestamp))
        
#         # Commit changes to the database
#         conn.commit()
#         print("Data saved to database")
# except Exception as e:
#     print("Error:", e)
#     # Rollback changes if an error occurs
#     conn.rollback()

In [ ]:
sql_query = "INSERT INTO sensor_b (water_post_code, sensor_b_temperature, sensor_b_humidity, sensor_b_scan_water_height, sensor_b_raindrop, sensor_b_status, created_at) VALUES (%s, %s, %s, %s, %s, %s, %s)"

print(sql_query)

try:
    with conn.cursor() as cursor:

        cursor.execute(sql_query, (post_code, temperature, humidity, sensor_b_scan_water_height, rain_value, point_b_status, timestamp))
        
        # Commit changes to the database
        conn.commit()
        print("Data saved to database")
except Exception as e:
    print("Error:", e)
    # Rollback changes if an error occurs
    conn.rollback()

In [ ]:
# sql_query = "INSERT INTO raindropsensor (sensor_post_code, raindrop_value, created_at) VALUES (%s, %s, %s)"

# print(sql_query)

# try:
#     with conn.cursor() as cursor:

#         cursor.execute(sql_query, (post_code, rain_value, timestamp))
        
#         # Commit changes to the database
#         conn.commit()
#         print("Data saved to database")
# except Exception as e:
#     print("Error:", e)
#     # Rollback changes if an error occurs
#     conn.rollback()